In [25]:
import pandas as pd
import numpy as np

In [26]:
station = pd.read_excel('ETC영업소주소.xlsx') # addresses added to ETC_영업소주소.xlsx
station.head()

,영업소코드,영업소명,세부주소1,세부주소2
0,76,가덕,부산광역시,부산광역시
1,246,가락,부산광역시,부산광역시
2,29,가락(개),부산광역시,부산광역시
3,977,가락(경),부산광역시,부산광역시
4,446,가락(특),부산광역시,부산광역시


In [27]:
# the API takes too long to load, use xlsx files instead
# data obtained at 7am (the busiest hour) and concerns suv only
# namely, 집계시=7 & TCS차종구분코드=1
trans0604 = pd.read_excel('영업소간통행시간240604_reduced.xlsx')
trans0604 = trans0604.sort_values(by=['출발영업소코드','도착영업소코드'],ignore_index=True)
trans0604 = trans0604.rename(columns={'통행시간':'통행시간04'})
trans0604.head()

,집계시,출발영업소코드,도착영업소코드,TCS차종구분코드,통행시간04
0,7,101,102,1,651
1,7,101,103,1,623
2,7,101,105,1,794
3,7,101,106,1,1210
4,7,101,107,1,1919


In [28]:
# load the others
trans0612 = pd.read_excel('영업소간통행시간240612_reduced.xlsx')
trans0612 = trans0612.sort_values(by=['출발영업소코드','도착영업소코드'],ignore_index=True)
trans0612 = trans0612.rename(columns={'통행시간':'통행시간12'})
trans0612.head()

,집계시,출발영업소코드,도착영업소코드,TCS차종구분코드,통행시간12
0,7,101,102,1,630
1,7,101,103,1,576
2,7,101,104,1,8433
3,7,101,105,1,725
4,7,101,106,1,1127


In [29]:
trans0620 = pd.read_excel('영업소간통행시간240620_reduced.xlsx')
trans0620 = trans0620.sort_values(by=['출발영업소코드','도착영업소코드'],ignore_index=True)
trans0620 = trans0620.rename(columns={'통행시간':'통행시간20'})
trans0620.head()

,집계시,출발영업소코드,도착영업소코드,TCS차종구분코드,통행시간20
0,7,101,102,1,653
1,7,101,103,1,617
2,7,101,104,1,8789
3,7,101,105,1,831
4,7,101,106,1,1280


In [30]:
trans0624 = pd.read_excel('영업소간통행시간240624_reduced.xlsx')
trans0624 = trans0624.sort_values(by=['출발영업소코드','도착영업소코드'],ignore_index=True)
trans0624 = trans0624.rename(columns={'통행시간':'통행시간24'})
trans0624.tail()

,집계시,출발영업소코드,도착영업소코드,TCS차종구분코드,통행시간24
18291,7,987,982,1,2717
18292,7,987,983,1,2291
18293,7,987,984,1,1368
18294,7,987,985,1,637
18295,7,987,986,1,511


In [31]:
# create a new df containing the list of avg values
trans = trans0604.copy()
trans = pd.merge(trans, trans0612, how='outer')
trans = pd.merge(trans, trans0620, how='outer')
trans = pd.merge(trans, trans0624, how='outer')
trans

,집계시,출발영업소코드,도착영업소코드,TCS차종구분코드,통행시간04,통행시간12,통행시간20,통행시간24
0,7,101,102,1,651.0,630.0,653.0,780.0
1,7,101,103,1,623.0,576.0,617.0,807.0
2,7,101,104,1,NaN,8433.0,8789.0,NaN
3,7,101,105,1,794.0,725.0,831.0,1045.0
4,7,101,106,1,1210.0,1127.0,1280.0,1577.0
...,...,...,...,...,...,...,...,...
24816,7,987,982,1,2807.0,2946.0,2979.0,2717.0
24817,7,987,983,1,2410.0,2263.0,2267.0,2291.0
24818,7,987,984,1,1441.0,1343.0,1352.0,1368.0
24819,7,987,985,1,607.0,618.0,624.0,637.0


In [32]:
# drop redundant columns
trans = trans.drop(columns=['집계시','TCS차종구분코드'])
trans

,출발영업소코드,도착영업소코드,통행시간04,통행시간12,통행시간20,통행시간24
0,101,102,651.0,630.0,653.0,780.0
1,101,103,623.0,576.0,617.0,807.0
2,101,104,NaN,8433.0,8789.0,NaN
3,101,105,794.0,725.0,831.0,1045.0
4,101,106,1210.0,1127.0,1280.0,1577.0
...,...,...,...,...,...,...
24816,987,982,2807.0,2946.0,2979.0,2717.0
24817,987,983,2410.0,2263.0,2267.0,2291.0
24818,987,984,1441.0,1343.0,1352.0,1368.0
24819,987,985,607.0,618.0,624.0,637.0


In [33]:
# take the average of the four transition time with NaNs ignored
tempZip = list(zip(trans['통행시간04'].values,trans['통행시간12'].values,
           trans['통행시간20'].values,trans['통행시간24'].values))
avgTrans = []
for i in range(len(tempZip)):
    avgTrans.append(np.nanmean(tempZip[i]))
trans['통행시간평균'] = pd.Series(avgTrans)
trans.head()

,출발영업소코드,도착영업소코드,통행시간04,통행시간12,통행시간20,통행시간24,통행시간평균
0,101,102,651.0,630.0,653.0,780.0,678.50
1,101,103,623.0,576.0,617.0,807.0,655.75
2,101,104,NaN,8433.0,8789.0,NaN,8611.00
3,101,105,794.0,725.0,831.0,1045.0,848.75
4,101,106,1210.0,1127.0,1280.0,1577.0,1298.50


In [34]:
trans = trans.drop(columns=['통행시간04','통행시간12','통행시간20','통행시간24'])
trans.head()

,출발영업소코드,도착영업소코드,통행시간평균
0,101,102,678.50
1,101,103,655.75
2,101,104,8611.00
3,101,105,848.75
4,101,106,1298.50


In [35]:
# merge the two dfs with respect to station code
mdf = pd.merge(trans, station, left_on='출발영업소코드', right_on='영업소코드').drop('영업소코드', axis=1)
mdf = pd.merge(mdf, station, left_on='도착영업소코드', right_on='영업소코드').drop('영업소코드', axis=1)
mdf.head()

,출발영업소코드,도착영업소코드,통행시간평균,영업소명_x,세부주소1_x,세부주소2_x,영업소명_y,세부주소1_y,세부주소2_y
0,101,102,678.50,서울,서울특별시,서울특별시,동수원,경기도,경기도
1,101,103,655.75,서울,서울특별시,서울특별시,수원신갈,경기도,경기도
2,101,104,8611.00,서울,서울특별시,서울특별시,지곡,경상남도,함양군
3,101,105,848.75,서울,서울특별시,서울특별시,기흥,경기도,경기도
4,101,106,1298.50,서울,서울특별시,서울특별시,오산,경기도,경기도


In [36]:
# take the same steps as in p4 to get an associate adjacency matrix
mdf = mdf.rename(columns={'세부주소1_x':'출발주소1', '세부주소2_x':'출발주소2',
                         '세부주소1_y':'도착주소1', '세부주소2_y':'도착주소2',
                         '영업소명_x':'출발영업소','영업소명_y':'도착영업소'})
mdf.head()

,출발영업소코드,도착영업소코드,통행시간평균,출발영업소,출발주소1,출발주소2,도착영업소,도착주소1,도착주소2
0,101,102,678.50,서울,서울특별시,서울특별시,동수원,경기도,경기도
1,101,103,655.75,서울,서울특별시,서울특별시,수원신갈,경기도,경기도
2,101,104,8611.00,서울,서울특별시,서울특별시,지곡,경상남도,함양군
3,101,105,848.75,서울,서울특별시,서울특별시,기흥,경기도,경기도
4,101,106,1298.50,서울,서울특별시,서울특별시,오산,경기도,경기도


In [37]:
mdf = mdf.drop(columns=['출발영업소코드','도착영업소코드'])
mdf.head()

,통행시간평균,출발영업소,출발주소1,출발주소2,도착영업소,도착주소1,도착주소2
0,678.50,서울,서울특별시,서울특별시,동수원,경기도,경기도
1,655.75,서울,서울특별시,서울특별시,수원신갈,경기도,경기도
2,8611.00,서울,서울특별시,서울특별시,지곡,경상남도,함양군
3,848.75,서울,서울특별시,서울특별시,기흥,경기도,경기도
4,1298.50,서울,서울특별시,서울특별시,오산,경기도,경기도


In [38]:
mdf = mdf.iloc[:,[1,2,3,4,5,6,0]]
mdf.head()

,출발영업소,출발주소1,출발주소2,도착영업소,도착주소1,도착주소2,통행시간평균
0,서울,서울특별시,서울특별시,동수원,경기도,경기도,678.50
1,서울,서울특별시,서울특별시,수원신갈,경기도,경기도,655.75
2,서울,서울특별시,서울특별시,지곡,경상남도,함양군,8611.00
3,서울,서울특별시,서울특별시,기흥,경기도,경기도,848.75
4,서울,서울특별시,서울특별시,오산,경기도,경기도,1298.50


In [39]:
# truncate the average values
import math
mdf['통행시간평균'] = mdf['통행시간평균'].map(lambda x:math.trunc(x))
mdf.head()

,출발영업소,출발주소1,출발주소2,도착영업소,도착주소1,도착주소2,통행시간평균
0,서울,서울특별시,서울특별시,동수원,경기도,경기도,678
1,서울,서울특별시,서울특별시,수원신갈,경기도,경기도,655
2,서울,서울특별시,서울특별시,지곡,경상남도,함양군,8611
3,서울,서울특별시,서울특별시,기흥,경기도,경기도,848
4,서울,서울특별시,서울특별시,오산,경기도,경기도,1298


In [40]:
# the first checkpoint - save mdf
mdf.to_csv('영업소간통행시간평균.csv')

In [41]:
# drop rows having nothing to do with 경상남도 by using 도착주소1 which refers to province
# namely, leave no rows whose start- and terminal stations are both outside 경상남도
dropCond = mdf[(mdf['출발주소1']!='경상남도') & (mdf['도착주소1']!='경상남도')].index
mdf = mdf.drop(dropCond)
mdf.head()

,출발영업소,출발주소1,출발주소2,도착영업소,도착주소1,도착주소2,통행시간평균
2,서울,서울특별시,서울특별시,지곡,경상남도,함양군,8611
32,서울,서울특별시,서울특별시,함안,경상남도,함안군,12835
33,서울,서울특별시,서울특별시,동창원,경상남도,창원시,13338
34,서울,서울특별시,서울특별시,서김해,경상남도,김해시,14776
35,서울,서울특별시,서울특별시,동김해,경상남도,김해시,12857


In [42]:
# then, first addresses are redundant
# a corresponding adjacency matrix will be constructed based on the second addresses
mdf = mdf.drop(columns=['출발주소1','도착주소1'])

# rename the second ones
mdf = mdf.rename(columns={'출발주소2':'출발영업소주소', '도착주소2':'도착영업소주소'})
mdf.head()

,출발영업소,출발영업소주소,도착영업소,도착영업소주소,통행시간평균
2,서울,서울특별시,지곡,함양군,8611
32,서울,서울특별시,함안,함안군,12835
33,서울,서울특별시,동창원,창원시,13338
34,서울,서울특별시,서김해,김해시,14776
35,서울,서울특별시,동김해,김해시,12857


In [43]:
# the second checkpoint - save mdf
mdf.to_csv('경남영업소간통행시간평균.csv', index=False) 

In [ ]:
# end of code